In [365]:
import tensorflow as tf
from tensorflow.keras.layers import Dense

import datetime
import shutil

print("TensorFlow version:", tf.__version__)

# Load the TensorBoard notebook extension
%load_ext tensorboard


TensorFlow version: 2.16.1
The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [366]:
 %reload_ext tensorboard
from tensorflow.keras import Model
from pathlib import Path
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint
from sklearn.metrics import mean_absolute_error, mean_squared_error
from joblib import load
from sklearn.preprocessing import StandardScaler

In [367]:
# Гиперпараметры
input_dim = 4  # Размерность входных данных
output_dim = 1  # Размерность выходных данных
hidden_layers = [32,64,64,32]  # Количество нейронов в каждом скрытом слое
activation = 'sigmoid'  # Функция активации для скрытых слоев
# activation = 'relu'  # Функция активации для скрытых слоев
learning_rate = 0.001 # /2 better
epochs = 1500

BATCH_SIZE = 256
BUFFER_SIZE = 512

In [368]:
input_dir = Path('./data/prepared')
logs_path = Path('./data/logs') / datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
model_path = Path('./data/models/nn')
if logs_path.exists():
  shutil.rmtree(logs_path)
logs_path.mkdir(parents=True)

X_train_name = input_dir / 'X_train.csv'
y_train_name = input_dir / 'y_train.csv'
X_test_name = input_dir / 'X_test.csv'
y_test_name = input_dir / 'y_test.csv'
X_val_name = input_dir / 'X_val.csv'
y_val_name = input_dir / 'y_val.csv'

X_train = pd.read_csv(X_train_name)
y_train = pd.read_csv(y_train_name)
X_test = pd.read_csv(X_test_name)
y_test = pd.read_csv(y_test_name)
X_val = pd.read_csv(X_val_name)
y_val = pd.read_csv(y_val_name)

In [369]:
with open(logs_path / 'params.txt', 'w') as file:
    file.write(f"input_dim = {input_dim}  # Размерность входных данных\n")
    file.write(f"output_dim = {output_dim}  # Размерность выходных данных\n")
    file.write(f"hidden_layers = {hidden_layers}  # Количество нейронов в каждом скрытом слое\n")
    file.write(f"activation = '{activation}'  # Функция активации для скрытых слоев\n")
    file.write(f"learning_rate = {learning_rate}\n")
    file.write(f"epochs = {epochs}\n\n")
    file.write(f"BATCH_SIZE = {BATCH_SIZE}\n")
    file.write(f"BUFFER_SIZE = {BUFFER_SIZE}\n")

In [370]:
train_ds = tf.data.Dataset.from_tensor_slices(
    (X_train, y_train)).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
test_ds = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(BATCH_SIZE)

In [371]:
model = Sequential(name="ModelForTGroup")
model.add(Input(shape=(input_dim, ), name=f"input_layer"))
for i, units in enumerate(hidden_layers, 1):
    # model.add(Dropout(0.01, trainable=True))
    model.add(Dense(units, activation=activation, name=f"hidden_layer_{i}"))
model.add(Dense(output_dim, name="output_layer"))

In [372]:
model.summary()

Model: "ModelForTGroup"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ hidden_layer_1 (Dense)          │ (None, 32)             │           160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ hidden_layer_2 (Dense)          │ (None, 64)             │         2,112 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ hidden_layer_3 (Dense)          │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ hidden_layer_4 (Dense)          │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ output_layer (Dense)            │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 8,545 (33.38 KB)

 Trainable params: 8,545 (33.38 KB)

 Non-trainable params: 0 (0.00 B)

In [373]:
model.compile(optimizer=Adam(learning_rate=learning_rate), metrics=['mae', 'mse'], loss='mse')

In [374]:
tensorboard_callback = TensorBoard(log_dir=logs_path, histogram_freq=1)
checkpoint_callback = ModelCheckpoint(filepath=model_path / f'{model.name}.keras', save_weights_only=False, save_best_only=True)

In [375]:
# Обучение модели
model.fit(X_train, y_train, batch_size=BATCH_SIZE, epochs=epochs, validation_data= (X_test, y_test), callbacks=[tensorboard_callback, checkpoint_callback])

Epoch 1/1500


3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 143ms/step - loss: 1.1490 - mae: 0.9266 - mse: 1.1490 - val_loss: 0.9986 - val_mae: 0.8204 - val_mse: 0.9986
Epoch 2/1500
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step - loss: 1.0727 - mae: 0.8147 - mse: 1.0727 - val_loss: 0.9577 - val_mae: 0.7228 - val_mse: 0.9577
Epoch 3/1500
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - loss: 1.0095 - mae: 0.7179 - mse: 1.0095 - val_loss: 0.9727 - val_mae: 0.6961 - val_mse: 0.9727
Epoch 4/1500
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step - loss: 1.0810 - mae: 0.7201 - mse: 1.0810 - val_loss: 0.9634 - val_mae: 0.7049 - val_mse: 0.9634
Epoch 5/1500
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - loss: 1.0900 - mae: 0.7396 - mse: 1.0900 - val_loss: 0.9547 - val_mae: 0.7361 - val_mse: 0.9547
Epoch 6/1500
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - loss: 1.0153 - mae: 0.7519 - mse: 1.0153 - val_loss: 0.9649 - val_mae: 0.7732 - val_mse: 0.9649
Epoch 7/1500
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - loss: 1.0359 - mae: 0.7818 - mse: 1.0359 - val_loss: 0.9771 - val

In [414]:
scaler = load(input_dir / 'scaler.pkl')
predictions = model.predict(X_val)
y_val = scaler.inverse_transform(y_val)
predictions = scaler.inverse_transform(predictions)
mae = mean_absolute_error(y_val, predictions)
mse = mean_squared_error(y_val, predictions)
print("Model MAE: ", mae)
print("Model MSE: ", mse)


6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
Model MAE:  8.797894300193269e+150
Model MSE:  1.5015800026795081e+302


In [377]:
%tensorboard --logdir data/logs

Reusing TensorBoard on port 6006 (pid 67712), started 11 days, 10:21:55 ago. (Use '!kill 67712' to kill it.)

In [378]:
str(logs_path.parent.absolute())

'd:\\22\\AICourseTGroup\\Sladkov\\data\\logs'